# Sistema de recomendação de imóveis - Parte 01
# Real state recomendation system  - Part 01

## Configurações básicas / Basic Configuration

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 6.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=ce75f843a3f5530011411a4ec9197e2202da836586c3e357baffb59709949980
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Alura Challange Semana 01") \
    .getOrCreate()

spark

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Carregando os dados / Loading the data

In [5]:
import pyspark.sql.functions as f
from pyspark.sql.types import IntegerType, StringType

In [6]:
dados = spark.read.json('/content/drive/MyDrive/Alura/Challenge Data Science/dataset_bruto.json')

In [7]:
dados.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

In [8]:
dados.show(20, False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------

# Preparação dos dados / Data preparation

In [9]:
dados_anuncio = dados\
  .select('anuncio.*')

In [10]:
dados_anuncio.show(20, False)

+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                |endereco                                                                                                                     |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                     |
+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+

In [11]:
dados_anuncio = dados_anuncio\
  .select("*")\
  .where("tipo_uso = 'Residencial'")\
  .where("tipo_unidade = 'Apartamento'")\
  .where("tipo_anuncio = 'Usado'")

dados_anuncio.show(20, False)

+-----+----------+---------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                                   |endereco                                                                                                                             |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                     |
+-----+----------+---------+---------+--------------------------------------------------------------------------

In [12]:


dados_anuncio = dados_anuncio\
  .withColumn('area_total', dados_anuncio['area_total'][0].cast(IntegerType()))\
  .withColumn('area_util', dados_anuncio['area_util'][0].cast(IntegerType()))\
  .withColumn('banheiros', dados_anuncio['banheiros'][0].cast(IntegerType()))\
  .withColumn('quartos', dados_anuncio['quartos'][0].cast(IntegerType()))\
  .withColumn('suites', dados_anuncio['suites'][0].cast(IntegerType()))\
  .withColumn('vaga', dados_anuncio['vaga'][0].cast(IntegerType()))

dados_anuncio.show(20, False)

+-----+----------+---------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                                   |endereco                                                                                                                             |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                     |
+-----+----------+---------+---------+--------------------------------------------------------------------------

In [13]:
dados_anuncio = dados_anuncio\
  .select("*", "endereco.bairro", "endereco.zona")\
  .drop("endereco")

dados_anuncio.show(20, False)

+-----+----------+---------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+------------------------+----------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                                   |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                     |bairro                  |zona      |
+-----+----------+---------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------

In [14]:
dados_anuncio = dados_anuncio\
  .select(
      "*",
      f.col("valores.iptu").alias("valor_iptu"),
      f.col("valores.condominio").alias("valor_condominio"),
      f.col("valores.tipo").alias("valor_tipo"),
      f.col("valores.valor").alias("valor_valor")
  )\
  .drop("valores")\
  

dados_anuncio.show(20, False)

+-----+----------+---------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+------------------------+----------+----------+----------------+----------+-----------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                                   |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|bairro                  |zona      |valor_iptu|valor_condominio|valor_tipo|valor_valor|
+-----+----------+---------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-

In [15]:
dados_anuncio = dados_anuncio\
  .withColumn('valor_valor', dados_anuncio['valor_valor'][0].cast(IntegerType()))\

dados_anuncio.show(20, False)

+-----+----------+---------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+------------------------+----------+----------+----------------+----------+-----------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                                   |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|bairro                  |zona      |valor_iptu|valor_condominio|valor_tipo|valor_valor|
+-----+----------+---------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-

In [16]:
dados_anuncio = dados_anuncio\
  .select("*")\
  .where(f.array_contains(dados_anuncio['valor_tipo'], 'Venda'))\

dados_anuncio.show(50, False)

+-----+----------+---------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+------------------------+----------+------------+----------------+----------------+-----------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                                   |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|bairro                  |zona      |valor_iptu  |valor_condominio|valor_tipo      |valor_valor|
+-----+----------+---------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+-----------

In [17]:
dados_anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- valor_iptu: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- valor_condominio: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- valor_tipo: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- valor_valor: integer (nullable = true)



## Salvando e lendo arquivos / Saving and reading files

In [18]:
dados_anuncio.write.parquet('/content/drive/MyDrive/Alura/Challenge Data Science/parquet', mode='overwrite')

In [19]:
dados_anuncio\
  .withColumn('caracteristicas', dados_anuncio['caracteristicas'].cast(StringType()))\
  .withColumn('valor_iptu', dados_anuncio['valor_iptu'].cast(StringType()))\
  .withColumn('valor_condominio', dados_anuncio['valor_condominio'].cast(StringType()))\
  .withColumn('valor_tipo', dados_anuncio['valor_tipo'].cast(StringType()))\
  .write.csv('/content/drive/MyDrive/Alura/Challenge Data Science/csv', mode='overwrite')

In [20]:
%%time
dados_parquet = spark.read.parquet('/content/drive/MyDrive/Alura/Challenge Data Science/parquet')

CPU times: user 6.35 ms, sys: 1.8 ms, total: 8.15 ms
Wall time: 255 ms


In [21]:
%%time
dados_csv = spark.read.csv('/content/drive/MyDrive/Alura/Challenge Data Science/csv')

CPU times: user 4.66 ms, sys: 1.13 ms, total: 5.79 ms
Wall time: 443 ms
